In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import tensorflow.keras
import os
checkpoint_path = "training_ResNet_52/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
from keras.callbacks import History

In [2]:
d_rate=0.1 #The rate of droupout for the layers in the ResNet
act='gelu'#The activation function use ('relu','swish','elu','gelu','selu')
lr=0.01 #The default learning rate Default 0.01
mom=0.9 #The momentum parameter Default 0.9
test_split=1 #Default 1 and range[1,0)
#Reducing test_split will make the model faster but will also reduce the accuracy of the Validation metric
#On changing test_split run the lower initialisation cell, marked below**

In [3]:
df_data=pd.read_csv("DataSetNormalised.csv")

df_data

In [4]:
df_label=pd.read_csv("LabelSet.csv")

In [5]:
x=np.array(df_data.iloc[:,1:])

In [6]:
y=np.array(df_label.iloc[:,1])

In [7]:
x=x.astype(np.float32)

In [8]:
#On changing test_split run this--------**
size=10000000 + test_split*1000000
x_train=x[:][:10000000]
y_train=y[:][:10000000]
x_test=x[:][10000000:size]
y_test=y[:][10000000:size]

y_train.shape

In [9]:
def identity_block(X,no_nodes,Activation):#A custom Residual identity block with a skip connection
    X_shortcut=X
    #There will be 3 layers in total for the sequential connection
    #Activation will be specified by the user as per the Keras Documentation
    #no_nodes denote the size of the model which will be built
    #Applying Batch Normalisation to size the inputs
    #Layer 1 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 2-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 3-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 4-------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X])
    X=tf.keras.layers.Activation('relu')(X)
    #--------------------------------------------------------
    X_shortcut2=X
     #Layer 5 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 6-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 7-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X,X_shortcut2])
    #Layer 8-------------------------------------------------
    X=tf.keras.layers.Activation('relu')(X)
    return X
    

In [10]:
inputs=tf.keras.layers.Input(shape=(28,))                    #Keras input layer
X=tf.keras.layers.Dense(300,activation=act)(inputs)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)        
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(X)     #keras output layer
model=tf.keras.Model(inputs,outputs)                         #Model declaration

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 300)          8700        ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 300)          90300       ['dense[0][0]']                  
                                                                                                  
 batch_normalization (BatchNorm  (None, 300)         1200        ['dense_1[0][0]']                
 alization)                                                                                   

In [12]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.01,
        beta_1=mom,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)



Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD502BCD0> and <keras.layers.core.dropout.Dropout object at 0x0000020BD507DBE0>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020BD5595A90> and <keras.layers.core.dense.Dense object at 0x0000020BD502BCD0>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD55A2E50> and <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020BD5595A90>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD55B7790> and <keras.layers.core.dropout.Dropout object at 0x0000020BD55AA190>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020BD55B7D30> and <keras.layers.mer


Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD586CAC0> and <keras.layers.merge.Add object at 0x0000020BD5600880>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020BD560C670> and <keras.layers.core.activation.Activation object at 0x0000020BD560C700>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD5881730> and <keras.layers.core.dense.Dense object at 0x0000020BD55C1280>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020BD55C69D0> and <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020BD58631C0>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD5600250> and <keras.layers.core.dro


Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020BD591BB80> and <keras.layers.core.dropout.Dropout object at 0x0000020BD55BD970>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD5909640> and <keras.layers.core.dense.Dense object at 0x0000020BD55B0730>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD5911FA0> and <keras.layers.core.dense.Dense object at 0x0000020BD55B72B0>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000020BD58E8E80> and <keras.layers.core.dropout.Dropout object at 0x0000020BD5899310>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x0000020BD5B455B0> and <keras.layers.merge.Add object at 0x0000020BD58998B0>

In [14]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=True,
                                                 save_best_only=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy",
                                                 factor=0.5,
                                                 patience=5,
                                                 min_lr=10**-6,
                                                 cooldown=5,
                                                 verbose = True)

history = History()

model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=10240,
    validation_data=(x_test,y_test),
    callbacks=[cp_callback, reduce_lr, history]
)

Epoch 1/300
977/977 [==============================] - ETA: 0s - loss: 0.6445 - accuracy: 0.6775 - auc: 0.7396
Epoch 00001: val_loss improved from inf to 0.55846, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 254s 249ms/step - loss: 0.6445 - accuracy: 0.6775 - auc: 0.7396 - val_loss: 0.5585 - val_accuracy: 0.7156 - val_auc: 0.7935 - lr: 0.0100
Epoch 2/300
977/977 [==============================] - ETA: 0s - loss: 0.5184 - accuracy: 0.7372 - auc: 0.8178
Epoch 00002: val_loss improved from 0.55846 to 0.51046, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 245s 251ms/step - loss: 0.5184 - accuracy: 0.7372 - auc: 0.8178 - val_loss: 0.5105 - val_accuracy: 0.7453 - val_auc: 0.8283 - lr: 0.0100
Epoch 3/300
977/977 [==============================] - ETA: 0s - loss: 0.5009 - accuracy: 0.7496 - auc: 0.8319
Epoch 00003: val_loss did not improve from 0.51046
977/977 [==============================] - 243s 249ms/step - l

Epoch 24/300
977/977 [==============================] - ETA: 0s - loss: 0.4611 - accuracy: 0.7754 - auc: 0.8605
Epoch 00024: val_loss did not improve from 0.45800
977/977 [==============================] - 241s 247ms/step - loss: 0.4611 - accuracy: 0.7754 - auc: 0.8605 - val_loss: 442897.3750 - val_accuracy: 0.7785 - val_auc: 0.8639 - lr: 0.0025
Epoch 25/300
977/977 [==============================] - ETA: 0s - loss: 0.4607 - accuracy: 0.7756 - auc: 0.8608
Epoch 00025: val_loss did not improve from 0.45800
977/977 [==============================] - 241s 247ms/step - loss: 0.4607 - accuracy: 0.7756 - auc: 0.8608 - val_loss: 33235716.0000 - val_accuracy: 0.7780 - val_auc: 0.8632 - lr: 0.0025
Epoch 26/300
977/977 [==============================] - ETA: 0s - loss: 0.4602 - accuracy: 0.7760 - auc: 0.8611
Epoch 00026: val_loss did not improve from 0.45800
977/977 [==============================] - 241s 247ms/step - loss: 0.4602 - accuracy: 0.7760 - auc: 0.8611 - val_loss: 197.1743 - val_accur

Epoch 48/300
977/977 [==============================] - ETA: 0s - loss: 0.4515 - accuracy: 0.7813 - auc: 0.8669
Epoch 00048: val_loss did not improve from 0.45800
977/977 [==============================] - 241s 247ms/step - loss: 0.4515 - accuracy: 0.7813 - auc: 0.8669 - val_loss: 318.4505 - val_accuracy: 0.7828 - val_auc: 0.8684 - lr: 3.1250e-04
Epoch 49/300
977/977 [==============================] - ETA: 0s - loss: 0.4512 - accuracy: 0.7815 - auc: 0.8670
Epoch 00049: val_loss did not improve from 0.45800
977/977 [==============================] - 241s 247ms/step - loss: 0.4512 - accuracy: 0.7815 - auc: 0.8670 - val_loss: 274.0798 - val_accuracy: 0.7833 - val_auc: 0.8687 - lr: 3.1250e-04
Epoch 50/300
977/977 [==============================] - ETA: 0s - loss: 0.4512 - accuracy: 0.7816 - auc: 0.8670
Epoch 00050: val_loss did not improve from 0.45800
977/977 [==============================] - 242s 247ms/step - loss: 0.4512 - accuracy: 0.7816 - auc: 0.8670 - val_loss: 457.1593 - val_accur

Epoch 72/300
977/977 [==============================] - ETA: 0s - loss: 0.4496 - accuracy: 0.7826 - auc: 0.8681
Epoch 00072: val_loss did not improve from 0.45800
977/977 [==============================] - 242s 247ms/step - loss: 0.4496 - accuracy: 0.7826 - auc: 0.8681 - val_loss: 109.8590 - val_accuracy: 0.7838 - val_auc: 0.8692 - lr: 7.8125e-05
Epoch 73/300
977/977 [==============================] - ETA: 0s - loss: 0.4496 - accuracy: 0.7824 - auc: 0.8681
Epoch 00073: val_loss did not improve from 0.45800
977/977 [==============================] - 241s 247ms/step - loss: 0.4496 - accuracy: 0.7824 - auc: 0.8681 - val_loss: 410.5022 - val_accuracy: 0.7837 - val_auc: 0.8692 - lr: 7.8125e-05
Epoch 74/300
977/977 [==============================] - ETA: 0s - loss: 0.4496 - accuracy: 0.7826 - auc: 0.8681
Epoch 00074: val_loss did not improve from 0.45800
977/977 [==============================] - 241s 247ms/step - loss: 0.4496 - accuracy: 0.7826 - auc: 0.8681 - val_loss: 34.0086 - val_accura

Epoch 96/300
977/977 [==============================] - ETA: 0s - loss: 0.4492 - accuracy: 0.7828 - auc: 0.8683
Epoch 00096: val_loss did not improve from 0.45800
977/977 [==============================] - 243s 249ms/step - loss: 0.4492 - accuracy: 0.7828 - auc: 0.8683 - val_loss: 495.3106 - val_accuracy: 0.7839 - val_auc: 0.8693 - lr: 9.7656e-06
Epoch 97/300
977/977 [==============================] - ETA: 0s - loss: 0.4492 - accuracy: 0.7826 - auc: 0.8683
Epoch 00097: val_loss did not improve from 0.45800
977/977 [==============================] - 243s 249ms/step - loss: 0.4492 - accuracy: 0.7826 - auc: 0.8683 - val_loss: 341.1631 - val_accuracy: 0.7839 - val_auc: 0.8693 - lr: 9.7656e-06
Epoch 98/300
 58/977 [>.............................] - ETA: 3:44 - loss: 0.4510 - accuracy: 0.7813 - auc: 0.8671

KeyboardInterrupt: 

In [15]:
model.save('ResNet_52')

with open("sunnyMamoni.txt", "w") as f:
    f.write(str(history.history))

INFO:tensorflow:Assets written to: ResNet_52\assets


C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
